<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/SVC/SVC_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification by SVC and Count Vectorizer**

----------

 import library

  إستيراد المكتبات

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from joblib import dump

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

**1- Import Data set**

In [ ]:
# df = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# df = pd.read_excel("drive/MyDrive/mining/3.xlsx")
df = pd.read_excel("drive/MyDrive/mining/4.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68690 entries, 0 to 68689
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    68312 non-null  object
 1   targe   68690 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


------
shuffle  
بعثرة البيانات

In [ ]:
df=shuffle(df)
df

,text,targe
45848,جددت نائبة رئيس المفوضية الأوروبية فيديريكا مو...,3
27980,عبدالاله بوسحابة اخبارنا المغربية يبدو أن رئيس...,3
49340,في ما يلي نتائج مباريات إياب الدور ربع النهائي...,4
266,علمت الصباح »، أن مسير الكباري »، الذي سيحتضن ...,0
4845,اصطدمت حافلة للنقل العمومي ببناية على مستوى شا...,1
...,...,...
44789,في تعليقه على مضمون البرقية الدبلوماسية السرية...,3
46642,قياديو العدالة والتنمية يصوبون مدفعيتهم تجاه و...,3
11455,أخبارنا المغربية مرة أخرى يطفو اسم عبد الله أف...,3
33785,بلغت إيرادات المغرب من عمليات تصوير أفلام أجنب...,0


------
**2- Clean Data**

تنظيف البيانات

In [ ]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

378
0
text     True
targe    True
dtype: bool
68690
                                                    text  targe
45848  جدت نائبه رئيس المفوضيه الاوروبيه فيديريكا موغ...      3
27980  عبدالاله بوسحابه اخبارنا المغربيه يبدو ان رئيس...      3
49340  في ما يلي نتائج مباريات اياب الدور ربع النهائي...      4
266    علمت الصباح » ان مسير الكباري » الذي سيحتضن حف...      0
4845   اصطدمت حافله لنقل العمومي بنايه علي مستوي شارع...      1
...                                                  ...    ...
44789  في تعليقه علي مضمون البرقيه الدبلوماسيه السريه...      3
46642  قياديو العداله والتنميه يصوبون مدفعيتهم تجاه و...      3
11455  اخبارنا المغربيه مره اخري يطفو اسم عبد اله افت...      3
33785  بلغت ايرادات المغرب من عمليات تصوير افلام اجنب...      0
25751  اخبارنا المغربيه سناء الوردي في اول رد فعل رسم...      2

[68312 rows x 2 columns]


----
 tokenizer
 
تقسيم كل نص على مستوى الكلمة

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
clean_df["text"] = clean_df["text"].apply(tokenizer.tokenize)
print(clean_df)

                                                    text  targe
45848  [جدت, نائبه, رئيس, المفوضيه, الاوروبيه, فيديري...      3
27980  [عبدالاله, بوسحابه, اخبارنا, المغربيه, يبدو, ا...      3
49340  [في, ما, يلي, نتائج, مباريات, اياب, الدور, ربع...      4
266    [علمت, الصباح, ان, مسير, الكباري, الذي, سيحتضن...      0
4845   [اصطدمت, حافله, لنقل, العمومي, بنايه, علي, مست...      1
...                                                  ...    ...
44789  [في, تعليقه, علي, مضمون, البرقيه, الدبلوماسيه,...      3
46642  [قياديو, العداله, والتنميه, يصوبون, مدفعيتهم, ...      3
11455  [اخبارنا, المغربيه, مره, اخري, يطفو, اسم, عبد,...      3
33785  [بلغت, ايرادات, المغرب, من, عمليات, تصوير, افل...      0
25751  [اخبارنا, المغربيه, سناء, الوردي, في, اول, رد,...      2

[68312 rows x 2 columns]


---
Delete StopWords

حذف الكلمات غير المؤثرة

In [ ]:
stopwords_list = stopwords.words('arabic')
listToStr = ' '.join([str(elem) for elem in stopwords_list]) 
print(listToStr)
#-------------
clean_df["text"] = clean_df["text"].apply(lambda x: [item for item in x if item not in stopwords_list])
((clean_df))

إذ إذا إذما إذن أف أقل أكثر ألا إلا التي الذي الذين اللاتي اللائي اللتان اللتيا اللتين اللذان اللذين اللواتي إلى إليك إليكم إليكما إليكن أم أما أما إما أن إن إنا أنا أنت أنتم أنتما أنتن إنما إنه أنى أنى آه آها أو أولاء أولئك أوه آي أي أيها إي أين أين أينما إيه بخ بس بعد بعض بك بكم بكم بكما بكن بل بلى بما بماذا بمن بنا به بها بهم بهما بهن بي بين بيد تلك تلكم تلكما ته تي تين تينك ثم ثمة حاشا حبذا حتى حيث حيثما حين خلا دون ذا ذات ذاك ذان ذانك ذلك ذلكم ذلكما ذلكن ذه ذو ذوا ذواتا ذواتي ذي ذين ذينك ريث سوف سوى شتان عدا عسى عل على عليك عليه عما عن عند غير فإذا فإن فلا فمن في فيم فيما فيه فيها قد كأن كأنما كأي كأين كذا كذلك كل كلا كلاهما كلتا كلما كليكما كليهما كم كم كما كي كيت كيف كيفما لا لاسيما لدى لست لستم لستما لستن لسن لسنا لعل لك لكم لكما لكن لكنما لكي لكيلا لم لما لن لنا له لها لهم لهما لهن لو لولا لوما لي لئن ليت ليس ليسا ليست ليستا ليسوا ما ماذا متى مذ مع مما ممن من منه منها منذ مه مهما نحن نحو نعم ها هاتان هاته هاتي هاتين هاك هاهنا هذا هذان هذه هذي هذين هكذا هل هلا هم هما هن هنا هنا

,text,targe
45848,"[جدت, نائبه, رئيس, المفوضيه, الاوروبيه, فيديري...",3
27980,"[عبدالاله, بوسحابه, اخبارنا, المغربيه, يبدو, ا...",3
49340,"[يلي, نتائج, مباريات, اياب, الدور, ربع, النهائ...",4
266,"[علمت, الصباح, ان, مسير, الكباري, سيحتضن, حفل,...",0
4845,"[اصطدمت, حافله, لنقل, العمومي, بنايه, علي, مست...",1
...,...,...
44789,"[تعليقه, علي, مضمون, البرقيه, الدبلوماسيه, الس...",3
46642,"[قياديو, العداله, والتنميه, يصوبون, مدفعيتهم, ...",3
11455,"[اخبارنا, المغربيه, مره, اخري, يطفو, اسم, عبد,...",3
33785,"[بلغت, ايرادات, المغرب, عمليات, تصوير, افلام, ...",0


---------
accuracy calculation function

عمل تابع لحساب الدقة

In [ ]:
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

----------------------
**3- Feature Extraction:** Count Vectorizer


استخراج الميزات(تحويل النص إلى أرقام)


**4- classification:** SVC

تصنيف البيانات

with Cross validation

لتوزيع عينات الاختبار والتأكد من عدم وجود

Overfitting

In [ ]:
%%time
from sklearn.feature_extraction.text import CountVectorizer

model = Pipeline([
                    ('count', CountVectorizer(max_features =10000)), # convert words to numbers using tfidf
                    ("clf", SVC()) # model the text
])

from sklearn.model_selection import KFold
kf = KFold(n_splits=4,shuffle=False)
result_Pipeline=[]
# for i in range(0,len(clean_df)):
y1 = clean_df['targe']
X1 = clean_df['text']

kf.split(X1)
j =0
for train_index, test_index in kf.split(X1):
#     # Split train-test
    X_train, X_test = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]
    model.fit(X_train.astype('str'), y_train)
    dump(model, "svm_model.joblib")
    
    y_pred=(model.predict(X_test.astype('str')))
    result_Pipeline.append(calculate_results(y_test, y_pred))

result_Pipeline

CPU times: user 2h 29min 4s, sys: 10 s, total: 2h 29min 14s
Wall time: 2h 28min 35s


حساب متوسط النتائج للـ5 مراحل من

 cross validation 

In [ ]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_Pipeline:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_Pipeline)
f1=f1/len(result_Pipeline)
precision=precision/len(result_Pipeline)
recall=recall/len(result_Pipeline)
# (result_DecisionTreeClassifier)
results_Pipeline=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_Pipeline

[{'accuracy': 93.64094156224382,
  'f1': 0.9365971780734839,
  'precision': 0.9369104136449349,
  'recall': 0.9364094156224383}]

--------
-----
---------
**5- Show Results**

النتائج

نتائج التدريب لكل قاعدة بيانات

In [ ]:
print('Pipeline results in dataset 1:')
results_Pipeline

Pipeline results in dataset 1:


[{'accuracy': 95.23069606378789,
  'f1': 0.9524673641161449,
  'precision': 0.9530264059889918,
  'recall': 0.9523069606378789}]

In [ ]:
print('Pipeline results in dataset 2:')
results_Pipeline

Pipeline results in dataset 2:


[{'accuracy': 95.32097910172035,
  'f1': 0.9533077743952252,
  'precision': 0.9536000855880794,
  'recall': 0.9532097910172036}]

In [ ]:
print('Pipeline results in dataset 3:')
results_Pipeline

Pipeline results in dataset 3:


[{'accuracy': 94.17067754648994,
  'f1': 0.9418666640934671,
  'precision': 0.9421649466933705,
  'recall': 0.9417067754648993}]

In [10]:
print('Pipeline results in dataset 4:')
results_Pipeline

Pipeline results in dataset 4:


[{'accuracy': 93.64094156224382,
  'f1': 0.9365971780734839,
  'precision': 0.9369104136449349,
  'recall': 0.9364094156224383}]

In [12]:
Final_Result=[
  {'accuracy': 95.23069606378789,
  'f1': 0.9524673641161449,
  'precision': 0.9530264059889918,
  'recall': 0.9523069606378789}
  ,
  {'accuracy': 95.32097910172035,
  'f1': 0.9533077743952252,
  'precision': 0.9536000855880794,
  'recall': 0.9532097910172036}
  ,
  {'accuracy': 94.17067754648994,
  'f1': 0.9418666640934671,
  'precision': 0.9421649466933705,
  'recall': 0.9417067754648993}
  ,
  {'accuracy': 93.64094156224382,
  'f1': 0.9365971780734839,
  'precision': 0.9369104136449349,
  'recall': 0.9364094156224383}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
print("RAM: 2.3G , CPU times: user 2h 29min 4s")
print('Final_Result:')
(Final_Result)

RAM: 2.3G , CPU times: user 2h 29min 4s
Final_Result:


[{'accuracy': 94.5908235685605,
  'f1': 0.9460597451695802,
  'precision': 0.9464254629788441,
  'recall': 0.945908235685605}]